<a href="https://colab.research.google.com/github/AlexeyTri/PyTorchTutorials/blob/main/Tutorial6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training pipeline

- prediction: PyTorch Model
- Gradients Computation: Autograd
- Loss Computation: PyTorch Loss
- Parameters updates: PyTorch Optimizer

1 - Desing model (input, output size, forward pass)

2 - Constractor loss and optimizer

3 - Training loop

    - forward pass: compute prediction

    - backward pass: gradients
    
    - update weights

In [44]:
import torch
import torch.nn as nn
import numpy as np
import sklearn
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [45]:
# prepare data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

In [46]:
n_samples, n_features = X.shape
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [47]:
# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test =sc.transform(X_test)

In [48]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [50]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [51]:
# 1) model
# f = wx + b, sigmoid at the end
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features,1)
    
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

model = LogisticRegression(n_features)

In [52]:
# 2) loss and optimizer
learning_rate = 0.01
# BCE - bynari cross entripy loss
criterion = nn.BCELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [53]:
# 3) training loop
num_epoch = 100
for epoch in range(num_epoch):
    # forward pass
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)

    # backward
    loss.backward()
    # updates
    optimizer.step()
    # zero_gradient
    optimizer.zero_grad()
    
    if (epoch+1) %10==0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

epoch: 10, loss = 0.4566
epoch: 20, loss = 0.3916
epoch: 30, loss = 0.3474
epoch: 40, loss = 0.3153
epoch: 50, loss = 0.2908
epoch: 60, loss = 0.2714
epoch: 70, loss = 0.2555
epoch: 80, loss = 0.2423
epoch: 90, loss = 0.2310
epoch: 100, loss = 0.2212
accuracy = 0.8947
